In [63]:
import json
import os
from datetime import datetime,timedelta
from crewai import Agent, Task, Crew, Process
from crewai_tools import CSVSearchTool
from dotenv import load_dotenv, find_dotenv
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchResults

In [64]:
from langchain_openai import ChatOpenAI

load_dotenv(find_dotenv())
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")



In [65]:
csv_imoveis = CSVSearchTool(
    csv_file="files/imoveis.csv"
)

In [66]:
# Agente corretor de imóveis

corretor_imoveis = Agent(
    role="Corretor de imóveis",
    goal="Obtenha as preferências do cliente e busque imóveis compatíveis no banco de dados.\n",
    backstory="Especialista no mercado imobiliário, encontra as melhores opções baseadas no perfil do cliente.\n",
    verbose=True,
    max_iter=5,
    tools=[csv_imoveis],
    allow_delegation=False,
    memory=True,
)


In [67]:
# Tarefa - Buscar imovies

buscar_imoveis = Task(
    description="Buscar imóveis compatíveis com as preferências do cliente",
    expected_output="Lista de imóveis compatíveis com as preferências do cliente com detalhes como preço, localização e características.",
    agent=corretor_imoveis
)

In [68]:
from crewai.tools import BaseTool

def obter_precos_imoveis(cidade:str = "geral"):
    precos = {
        "sao paulo":{"tendencia":"aumento", "percentual":5.05},
        "rio de janeiro":{"tendencia":"estabilidade", "percentual":2.02},
        "belo horizonte":{"tendencia":"queda", "percentual":3.03},
        "curitiba":{"tendencia":"estabilidade", "percentual":1.01},
        "fortaleza":{"tendencia":"queda", "percentual":0.04},
        "salvador":{"tendencia":"aumento", "percentual":0.03},
        "geral":{"tendencia":"aumento", "percentual":4.03},
    }
    return precos.get(cidade, precos["geral"])

class TendenciaPrecosImoveisTool(BaseTool):
    name:str = "Analisador de preços imobiliários"
    description:str = """
    Obtém tendências de precos de imoveis com base na cidade especificada.
    """

    def _run(self, cidade:str)-> dict:
        """
        Excuta a analise de precos imobiliários e retorna a tendencia com base na cidade.
        """
        try:
            return obter_precos_imoveis(cidade)
        except Exception as e:
            return {"error": f"Erro ao obter tendência de preços: {str(e)}"}

In [69]:
analista_mercado = Agent(
    role="Analista de mercado imobiliário",
    goal="Analisa tendencias de precos e ajuda a prever a valorização ou desvalorização dos imoveis na cidade {cidade}.\n",
    backstory="Experiente no mercado imobiliário, analisa as tendências de preços e fornece recomendações estratégicas para prever preços futuros.\n",
    verbose=True,
    max_iter=5,
    allow_delegation=False,
    memory=True,
)

In [70]:
obter_tendencias = Task(
    description="""
    Analise o historico de precos de imoveis na cidade {cidade} e forneça insights sobre a valorização\n
    ou desvalorização. Considere o tipo de imovel {tipo_imovel} e a faixa de preço {faixa_preco}.\n
    """,
    expected_output="Resumo da tentencias dos precos no mercado imobiliario.",
    tools=[TendenciaPrecosImoveisTool()],
    agent=analista_mercado,
    parameters=["cidade"]
)

In [71]:
analista_noticias = Agent(
    role="Analista de noticias imobiliárias",
    goal="Busca noticias relevantes osbre o mercado imobiliario para avaliar fatores externos.\n",
    backstory="Especialista em analisar noticias e tendencias economicas que afetam os precos dos imoveis\n",
    verbose=True,
    max_iter=5,
    memory=True,
)
    

In [72]:
searchTool = DuckDuckGoSearchResults(backend="news", num_results=5)


In [ ]:
searchTool

In [74]:
buscar_noticias = Task(
    description="""
    Busca as ultimas (recentes) noticias sobre o mercado imobiliário na cidade {cidade} na data atual: {datetime.now()}.
    """,
    expected_output="Resumo das principais noticias e tendencias imobiliárias encontradas.",
    tool=[searchTool],
    agent=analista_noticias,
)

In [75]:
consultor_financeiro = Agent(
    role="Consultor financeiro",
    goal="Analisa o impacto das noticias imobiliárias no mercado financeiro com base no perfil do cliente   .\n",
    backstory="Experiente em analisar noticias e tendencias economicas que afetam os precos dos imovei. Ajuda os clientes a escolher as melhores opções de investimento.\n",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True
)

In [76]:
calcular_financeiamento = Task(
    description="""
    Analise a renda do cliente e sugira opções de financiamento viavies.\n
    """,
    expected_output="Tabela comparativa com diferentes financiamentos, taxa de juros e prazos",
    agent=consultor_financeiro
)

In [77]:
redator = Agent(
    role="Redator de relatorios imobiliarios",
    goal="Cria relatorios detalhados e persuativos para os clientes com base nas informações coletadas.\n",
    backstory="Especialista em criar relatorios detalhados e personalizados para os clientes de forma clara e objetiva.\n",
    verbose=True,
    allow_delegation=False,
    max_iter=5,
    memory=True,
)

gerar_relatorio = Task(
    description="""
    Gere um realtorio detalhado sobre o melhor imovel encontrado, considerando precos, tendencias e financiamento.\n
    """,
    expected_output="Relatório formatado com resumo do mercado, opções recomendadas e justificativa da escolha.",
    agent=redator,
    context=[buscar_imoveis, obter_tendencias, buscar_noticias, calcular_financeiamento]
)

In [ ]:
crews = Crew(
    agents=[corretor_imoveis, analista_mercado, analista_noticias, consultor_financeiro, redator],
    tasks=[buscar_imoveis, obter_tendencias, buscar_noticias, calcular_financeiamento, gerar_relatorio],
    verbose=True,
    process=Process.hierarchical,
    full_output=True,
    share_crew=False,
    max_iter=15,
    manager_llm=llm
)

resultado = crews.kickoff(
    inputs={
        "cidade": "São Paulo",
        "tipo_imovel": "apartamento",
        "faixa_preco": "1000000-2000000"
    }
)

In [ ]:
print(resultado.raw)

from IPython.display import Markdown, display

display(Markdown(str(resultado.raw)))

with open('relatorio_imobiliario.pdf', 'w', encoding='utf-8') as file:
    file.write(str(resultado))

import markdown

html = markdown.markdown(str(resultado))
with open('relatorio_imobiliario.html', 'w', encoding='utf-8') as file:
    file.write(html)